In [ ]:
from TModel.TransformerModel import TranscriptionTransformerModel
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from Tokenizer.loaderH5 import H5GuitarTokenizer
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
from TUtils import random_string
from lightning import Trainer
import torch
# from TranscriptionDataset.TranscriptionDataset import TranscriptionDataset
from TranscriptionDataset import TranscriptionDataset
# torch.set_float32_matmul_precision('medium')

In [1]:
datasetLocation = "Trainsets/S_Tier_1695428558_mTokens1000_mNoS60.hdf5"
wandbProject = "TranscriptionModel"
batchSize = 10
num_workers = 4

In [ ]:
dataset,pipe = TranscriptionDataset.getDataPipe(datasetLocation,batchSize)
train_pipe,test_pipe = pipe.random_split({"train":0.8,"test":0.2},42)

train_dl = DataLoader(dataset=train_pipe, batch_size=batchSize,num_workers=num_workers)
test_dl = DataLoader(dataset=test_pipe, batch_size=batchSize,num_workers=num_workers)


In [ ]:
model = TranscriptionTransformerModel(dataset.getVocabSize(),d_ff=2048)

# wandb_logger = WandbLogger(project=wandbProject)
# wandb_logger.experiment.config.update(dataset.meta_data)
# wandb_logger.experiment.config["batchSize"] = batchSize
checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    dirpath=f'Models/GuitarTranscription/{random_string(10)}/',
    filename='GuitarTranscriptionModel-{epoch:02d}-{train_loss:.2f}',
    every_n_train_steps=1000,
    save_top_k=3,
    mode='min',
)
trainer = Trainer(
    default_root_dir='Models/',
    max_epochs=5,
    # profiler="simple",
    # logger=wandb_logger,
    callbacks=[checkpoint_callback],
    # max_time="00:00:05:00"
)

trainer.fit(model=model, train_dataloaders=train_dl)